In [1]:
##vocab production script
#imports
import multiprocessing
from multiprocessing import Pool
import math, random, sys
import pickle
import argparse
from functools import partial
import torch
import numpy
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdDepictor
from rdkit.Chem.Draw import rdMolDraw2D
from hgraph import MolGraph, common_atom_vocab, PairVocab
import rdkit
import rdkit.Chem as Chem
import networkx as nx
from hgraph.chemutils import *
from hgraph.nnutils import *
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import SVG
import time
from rdkit.Chem import Draw
import itertools
from collections import Counter

IPythonConsole.drawOptions.addAtomIndices = True
IPythonConsole.molSize = 300,300

/home/marcase/.conda/envs/hgraph-rdkit/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AttributeError: module 'rdkit.Chem.Draw.IPythonConsole' has no attribute 'drawOptions'

In [126]:
# peptide = 'OC(C[C@@H](C(N[C@H](C(N[C@H](C(N[C@H](C(NCC(N[C@H](C(N[C@@]([H])([C@@H](C)CC)C(N[C@@H](CC(C)C)C(N[C@H](C(N[C@H](C(N[C@@H](CC1=CC=CC=C1)C(N2CCC[C@H]2C(N[C@H](C(N[C@@]([C@H](O)C)(C(N[C@H](C(N[C@H](C(N[C@H](C(N[C@H](C(N[C@H](C(O)=O)CSC3)=O)CSCC3=O)=O)C(C)C)=O)CC4=CC=C(O)C=C4)=O)CC5=CNC6=C5C=CC=C6)=O)[H])=O)CO)=O)=O)=O)CCSC)=O)CCCCN[H])=O)=O)=O)CC7=CNC=N7)=O)=O)CCC(N)=O)=O)CCC(O)=O)=O)CS[H])=O)NC([C@@H](NC([C@@H](NC([C@H](C)NC(CN)=O)=O)CCCNC(N)=N)=O)CC(N)=O)=O)=O'
#c term proline
peptide = 'N[C@@H]([C@]([H])(O)C)C(=O)N[C@@H](CSCC(C3)=O)C(=O)N[C@@H](C(C)C)C(=O)N1[C@@H](CCC1)C(=O)N[C@@H](CCC(=O)O)C(=O)N[C@@H](CC1=CN=C-N1)C(=O)N[C@@H](CS3)C(=O)N[C@@H](CO)C(=O)N[C@@H](CSCC(C3)=O)C(=O)N[C@@H](Cc1ccc(O)cc1)C(=O)N[C@@H](CCCNC(=N)N)C(=O)N[C@@H](CS3)C(=O)N1[C@@H](CCC1)C(=O)O'
#n term proline
# peptide = 'N1[C@@H](CCC1)C(=O)N[C@@H](CSCC(C3)=O)C(=O)N[C@@H](CS3)C(=O)N[C@@H](C)C(=O)N[C@@H](CCCNC(=N)N)C(=O)N[C@@H](CSCC(C3)=O)C(=O)N1[C@@H](CCC1)C(=O)N[C@@H](CC(=CN2)C1=C2C=CC=C1)C(=O)N[C@@H](CC(=O)O)C(=O)N[C@@H](CC(=O)O)C(=O)N[C@@H](CS3)C(=O)N[C@@H](CCCNC(=N)N)C(=O)O'
peptide = get_mol(peptide)

In [109]:
def mol_with_atom_index(mol):
    for atom in mol.GetAtoms():
        atom.SetAtomMapNum(atom.GetIdx())
    return mol

mol_with_atom_index(peptide)
Draw.MolToFile(peptide, "peptide.png",size=(6000,6000))

In [131]:
#define matches for side chain traversal
match_molecule = 'NCC=O'
matches = peptide.GetSubstructMatches(Chem.MolFromSmiles(match_molecule),useChirality=True)
print(matches)

#define matches with cterm and overlap (for tree_decomp method)
match_molecule_with_cterm = 'O=CCNC'
matches_with_cterm = peptide.GetSubstructMatches(Chem.MolFromSmiles(match_molecule_with_cterm),useChirality=True)
# print(matches_with_cterm)

#define matches with nterm and overlap (for tree_decomp method)
match_molecule_with_nterm = 'NCC(N)=O'
matches_with_nterm = peptide.GetSubstructMatches(Chem.MolFromSmiles(match_molecule_with_nterm),useChirality=True)
# print(matches_with_nterm)

#backbone atoms
# backbone_atoms = np.hstack(matches_with_nterm)
backbone_atoms = np.hstack(np.unique(np.concatenate((matches_with_nterm,peptide.GetSubstructMatches(Chem.MolFromSmiles('NCC(O)=O'),useChirality=True)))))
sidechain_atoms = np.setdiff1d(np.array(range(1,peptide.GetNumAtoms())),backbone_atoms)
# print(backbone_atoms)
# print(sidechain_atoms)

#nterm cluster
n_term = [np.array(m) for m in matches_with_nterm if (sum([x in np.hstack(matches_with_cterm) for x in m]) < 3) if not any([x in sidechain_atoms for x in m])][0]
print(n_term)

#cterm cluster
c_term = [np.array(m) for m in matches_with_cterm if (sum([x in np.hstack(matches_with_nterm) for x in m]) < 3) if not any([x in sidechain_atoms for x in m])][0]
print(c_term)

#separately define proline, because the backbone creates double recognition
proline = 'O=CC1CCCN1'
proline_matches = peptide.GetSubstructMatches(Chem.MolFromSmiles(proline),useChirality=True)
proline_match_ids = np.hstack(proline_matches)
print(proline_matches)

#update cluster with correct proline cluster
clusters = [i for i in np.array(matches_with_nterm).reshape(len(matches_with_nterm),5)]
if any([x in c_term for x in proline_match_ids]): #c_term is proline
    proline_c_term = 'O=C(O)C1CCCN1'
    proline_c_term_match = peptide.GetSubstructMatches(Chem.MolFromSmiles(proline_c_term),useChirality=True)
    clusters.append(np.array([p for p in proline_c_term_match if any([x in c_term for x in p])][0]))
else:
    other_c_term = 'NCC(=O)O'
    other_c_term_match = peptide.GetSubstructMatches(Chem.MolFromSmiles(other_c_term),useChirality=True)
    # print(other_c_term_match)
    clusters.append(c_term)
if any([x in n_term for x in proline_match_ids]): #n_term is proline
    1
#replace non-terminal prolines with correct prolines   
clusters = [clusters[index] for index,match in enumerate(matches) if not any([x in proline_match_ids for x in match])]
# [clusters.append(np.array(proline_match)) for proline_match in proline_matches]

#identify c_alphas
c_alphas = np.reshape(matches,(len(matches),4))[:,1]
# print(c_alphas)


# backbone_atoms_with_overlap = np.hstack(np.array(matches_with_overlap)[:,[2,4]])
# backbone_atoms_with_overlap
# backbone_atoms_with_overlap = Counter(backbone_atoms_with_overlap)

# termini = list([atom for atom in backbone_atoms_with_overlap if (backbone_atoms_with_overlap[atom]==1) and (atom not in proline_match_ids)])
# print(termini)

# #assign n and c terminus based on # of bonds to other atoms
# if len(peptide.GetAtomWithIdx(int(termini[1])).GetNeighbors()) == 2:
#     n_terminal_atom = int(termini[1])
#     c_terminal_atom = int(termini[0])
# else:
#     c_terminal_atom = int(termini[1])
#     n_terminal_atom = int(termini[0])

# #extract atom indices of the n terminus
# n_terminus = np.array([],dtype=int)
# for neighbor in peptide.GetAtomWithIdx(n_terminal_atom).GetNeighbors():
#     if neighbor.GetIdx() not in backbone_atoms:
#         n_terminus = np.append(n_terminus, int(neighbor.GetIdx()))

# n_terminus = np.append(n_terminus, n_terminal_atom)
# clusters.append(n_terminus)

# #extract atom indices of the c terminus
# c_terminus = np.array([],dtype=int) 
# for neighbor in peptide.GetAtomWithIdx(c_terminal_atom).GetNeighbors():
#     if neighbor.GetIdx() not in backbone_atoms:
#         if 8 in [atom.GetAtomicNum() for atom in peptide.GetAtomWithIdx(neighbor.GetIdx()).GetNeighbors()]:
#             c_terminus = np.append(c_terminus, [int(new_neighbor.GetIdx()) for new_neighbor in peptide.GetAtomWithIdx(neighbor.GetIdx()).GetNeighbors()])
#             c_terminus = np.append(c_terminus, [neighbor.GetIdx()])
# clusters.append(c_terminus)

clusters

((0, 1, 5, 6), (7, 8, 15, 16), (17, 18, 22, 23), (24, 25, 29, 30), (31, 32, 38, 39), (40, 41, 48, 49), (50, 51, 54, 55), (56, 57, 60, 61), (62, 63, 70, 71), (72, 73, 82, 83), (84, 85, 93, 94), (95, 96, 99, 100), (101, 102, 106, 107))
[0 1 5 7 6]
[107 106 102 101  99]
((30, 29, 25, 26, 27, 28, 24), (107, 106, 102, 103, 104, 105, 101))


[array([0, 1, 5, 7, 6]),
 array([ 7,  8, 15, 17, 16]),
 array([17, 18, 22, 24, 23]),
 array([31, 32, 38, 40, 39]),
 array([40, 41, 48, 50, 49]),
 array([50, 51, 54, 56, 55]),
 array([56, 57, 60, 62, 61]),
 array([62, 63, 70, 72, 71]),
 array([72, 73, 82, 84, 83]),
 array([84, 85, 93, 95, 94]),
 array([ 95,  96,  99, 101, 100])]

In [106]:
other_c_term = 'NCC(=O)O'
other_c_term_match = peptide.GetSubstructMatches(Chem.MolFromSmiles(other_c_term),useChirality=True)
print(other_c_term_match)

((101, 102, 106, 107, 108),)


In [ ]:
##recursively identify alpha carbons, and map the atom indices that contain their side chains
c_alphas_neighbors = [peptide.GetAtomWithIdx(int(x)).GetNeighbors() for x in c_alphas]

for index,c_alpha in enumerate(c_alphas):
    c_alpha_neighbors = peptide.GetAtomWithIdx(int(c_alpha)).GetNeighbors()
    # print('starting c_alpha atom index: ' + str(c_alpha))
    # print('neighbor indices: ' +str([neighbor.GetIdx() for neighbor in c_alpha_neighbors]))
    # print('neighbor atomic masses: ' + str([neighbor.GetAtomicNum() for neighbor in c_alpha_neighbors]))
    # print('starting atoms: ' + str([c_alpha_neighbor.GetIdx() for c_alpha_neighbor in c_alpha_neighbors]))
    if c_alpha == c_terminal_atom:
        # print('c terminus skipped: ' + str(c_alpha))
        continue
    elif c_alpha == n_terminal_atom:
        # print('n terminus skipped: ' + str(c_alpha))
        continue
    if c_alpha in proline_match_ids:
        # print('skipped proline')
        continue
    for neighbor in c_alpha_neighbors:
        if (neighbor.GetIdx() not in backbone_atoms) and (neighbor.GetAtomicNum() == 6):
            # print('starting side chain atom: '+str(neighbor.GetIdx()) + ', atomic num: ' + str(neighbor.GetAtomicNum()))
            side_chain_atoms = np.array([neighbor.GetIdx()])
            new_neighbors = neighbor.GetNeighbors()
            # print('starting neighbor atom(s): ' + str([new_neighbor.GetIdx() for new_neighbor in new_neighbors]) + ', atomic num(s): ' + str([new_neighbor.GetAtomicNum() for new_neighbor in new_neighbors]))
            atoms_to_examine = np.array([])
            while(len(new_neighbors) > 0):
                for new_neighbor in new_neighbors:
                    if (new_neighbor.GetIdx() not in side_chain_atoms) and (new_neighbor.GetIdx() not in backbone_atoms):
                        # print('new neighbor atom found, index: ' + str(new_neighbor.GetIdx()))
                        side_chain_atoms = np.append(side_chain_atoms,new_neighbor.GetIdx())
                        # print('side_chain_atom(s): ' + str(side_chain_atoms))
                        atoms_to_examine = np.append(atoms_to_examine,new_neighbor.GetIdx())
                        # print('atom(s) to examine: ' + str(atoms_to_examine))
                    elif new_neighbor.GetIdx()  in side_chain_atoms:
                        # print('already found this atom in side chain: ' + str(new_neighbor.GetIdx()))
                        continue
                    elif new_neighbor.GetIdx() in backbone_atoms:
                        # print('already found this atom in back bone: ' + str(new_neighbor.GetIdx()))
                        continue
                if len(atoms_to_examine) == 0:
                    # print('no new atoms to explore')
                    break
                else:
                    new_neighbors = np.hstack([peptide.GetAtomWithIdx(int(atom)).GetNeighbors() for atom in atoms_to_examine])
                    # print('new neighbors to investigate: ' + str([new_neighbor.GetIdx() for new_neighbor in new_neighbors]))
                    atoms_to_examine = np.array([])
            # print('complete mapped side chain: ' + str([x for x in side_chain_atoms]))
            clusters.append(np.concatenate([np.array([x for x in side_chain_atoms]),np.array([c_alpha])]))
    #neighbors are not in a consistent order
# [peptide.GetAtomWithIdx(x).GetNeighbors() for x in c_alphas]

In [ ]:
#check if all atoms are in a cluster
test = np.hstack(clusters)
test = set(np.sort(test))
if len(test) == len(peptide.GetAtoms()):
    print('molecule clusters identified correctly')
else:
    print(test)

In [7]:
#get atom_cls
atom_cls = [[] for i in range(len(peptide.GetAtoms()))]
for i in range(len(clusters)):
    for atom in clusters[i]:
        atom_cls[atom].append(i)

In [128]:
clusters

[array([0, 1, 5, 7, 6]),
 array([ 7,  8, 15, 17, 16]),
 array([17, 18, 22, 24, 23]),
 array([31, 32, 38, 40, 39]),
 array([40, 41, 48, 50, 49]),
 array([50, 51, 54, 56, 55]),
 array([56, 57, 60, 62, 61]),
 array([62, 63, 70, 72, 71]),
 array([72, 73, 82, 84, 83]),
 array([84, 85, 93, 95, 94]),
 array([ 95,  96,  99, 101, 100])]

In [8]:
atom_cls

[[0],
 [],
 [],
 [],
 [0],
 [0],
 [0],
 [0, 1],
 [],
 [],
 [],
 [],
 [],
 [],
 [1],
 [1],
 [1],
 [1],
 [],
 [],
 [],
 [10],
 [10],
 [10],
 [2, 10],
 [10],
 [10],
 [10],
 [2],
 [2],
 [2],
 [2, 3],
 [],
 [],
 [],
 [],
 [],
 [3],
 [3],
 [3],
 [3, 4],
 [],
 [],
 [],
 [],
 [],
 [],
 [4],
 [4],
 [4],
 [4, 5],
 [],
 [],
 [5],
 [5],
 [5],
 [5, 6],
 [],
 [],
 [6],
 [6],
 [6],
 [6, 7],
 [],
 [],
 [],
 [],
 [],
 [],
 [7],
 [7],
 [7],
 [7, 8],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [8],
 [8],
 [8],
 [8, 9],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [9],
 [9],
 [9],
 [9],
 [],
 [],
 [11],
 [11],
 [11],
 [11],
 [11],
 [11],
 [11],
 [],
 [],
 []]

In [ ]:
matches_with_overlap